In [10]:
import torch
import torch.nn as nn
import numpy as np
import math

In [65]:
with open("../data/datasets/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [204]:
indices = np.random.permutation(all_data.shape[0])
split_idx = math.floor(all_data.shape[0] * 0.9)
train_idx, test_idx = indices[:split_idx], indices[split_idx:]
train_data, test_data = torch.from_numpy(all_data[train_idx,:]).type(torch.FloatTensor), torch.from_numpy(all_data[test_idx,:]).type(torch.FloatTensor)
train_labels, test_labels = torch.from_numpy(labels[train_idx]).type(torch.FloatTensor), torch.from_numpy(labels[test_idx]).type(torch.FloatTensor)


In [205]:
print(all_data)
print(labels)

[[0.02   0.0371 0.0428 ... 0.0084 0.009  0.0032]
 [0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 ...
 [0.0522 0.0437 0.018  ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049  ... 0.0079 0.0036 0.0048]
 [0.026  0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [199]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, input, labels, n_epochs, loss_fn, optimizer, batch_size=10):
        for epoch in range(n_epochs):
            start = 0
            #for i in range(math.floor(input.shape[0]/batch_size)):
            #    X_batch = input[start:(start+batch_size)]
            #    Ybatch = labels[start:(start+batch_size)]
            #    start += batch_size
            #    pass
            prediction = self.forward(input)
            loss = loss_fn(prediction, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch {epoch}:\tloss:{loss}")
            
    


In [200]:
model = Network(input_size=60, output_size=1)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.9, betas=(0.8, 0.95))
model.train()
model.fit(input=train_data, labels=train_labels, n_epochs=100, loss_fn=loss_fn, optimizer=optimizer)

Epoch 0:	loss:0.49889901280403137
Epoch 1:	loss:0.498904287815094
Epoch 2:	loss:0.49899664521217346
Epoch 3:	loss:0.4991132318973541
Epoch 4:	loss:0.4992375075817108
Epoch 5:	loss:0.4993674159049988
Epoch 6:	loss:0.4994971454143524
Epoch 7:	loss:0.49961763620376587
Epoch 8:	loss:0.49971503019332886
Epoch 9:	loss:0.49977847933769226
Epoch 10:	loss:0.49980172514915466
Epoch 11:	loss:0.4997945725917816
Epoch 12:	loss:0.49976563453674316
Epoch 13:	loss:0.4997183382511139
Epoch 14:	loss:0.49966076016426086
Epoch 15:	loss:0.499599426984787
Epoch 16:	loss:0.49954211711883545
Epoch 17:	loss:0.4994968771934509
Epoch 18:	loss:0.49946531653404236
Epoch 19:	loss:0.4994491636753082
Epoch 20:	loss:0.4994497299194336
Epoch 21:	loss:0.49946314096450806
Epoch 22:	loss:0.4994858205318451
Epoch 23:	loss:0.49951350688934326
Epoch 24:	loss:0.49954187870025635
Epoch 25:	loss:0.4995667040348053
Epoch 26:	loss:0.4995846748352051
Epoch 27:	loss:0.49959462881088257
Epoch 28:	loss:0.49959567189216614
Epoch 29:	l

In [198]:
model.eval()
model.forward(test_data)


tensor([[0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056],
        [0.5056]], grad_fn=<SigmoidBackward0>)